In [3]:
import BIG_HAMILTON as bh
import DCA_square_loop as DCA
import numpy as np
from math import factorial as fact
from scipy.sparse.linalg import eigs
import copy
import progressbar
def eigen_values_vectors(m,n_down,n_up,n_max,U=8,mu=0,t=0,V=0):
    U, mu, t_renorm, V_renorm, hybridization, gamma, ek, eq = DCA.get_start_parametrs(U,mu,t,V)
    H = bh.Hamiltonian(U, t_renorm, V_renorm, hybridization, gamma, ek, eq).matrix(m,m,m,n_down,n_up,n_max)
    E,V = eigs(H,k=10,which='SR')
    return E,V

def number_condition(m_c, m_d,m_b,n_down, n_up, n_max):
    '''Число состояний'''
    R_up = (fact(m_c+m_d)/fact(n_up)/fact(m_c+m_d-n_up))
    R_down = (fact(m_c+m_d)/fact(n_down)/fact(m_c+m_d-n_down))
    R_bose = (n_max+1)**m_b
    return R_up, R_down, R_bose

def create_dict(basis):
    '''Создаем словарь собственных функций'''
    key = [''.join(map(str,basis[i])) for i in range(basis.shape[0])]
    value = range(len(key))
    indexes = dict(zip(key,value))
    return indexes
def up(number_to_up, func): 
        '''Функция возвращает коэффициент и новую функцию'''
        function = copy.copy(func)
        if function[number_to_up] == 1:
            return [0,function]
        else:
            coef_up = np.sqrt(function[number_to_up] + 1)
            function[number_to_up] = function[number_to_up] + 1
            return [round(coef_up,5), function]
def down(number_to_down, func): 
        '''Функция возвращает коэффициент и новую функцию'''
        function = copy.copy(func)
        if function[number_to_down] == 0:
            return [0,function]
        else:
            coef_down = np.sqrt(function[number_to_down])
            function[number_to_down] = function[number_to_down] - 1
            return [round(coef_down,5), function]
# Знак 
def sign(right_edge,func, left_edge=0):
    '''Функция определяет знак перескока'''
    if sum(func[left_edge:right_edge])%2 == 0:
        return 1
    else:
        return -1
    
def brakets(dict_n, dict_n_1,basis_n, basis_n_1,i,j,Vn,Vn1,R_up_n, R_down_n, R_down_n_1, R_bose_n,col1,col2):
    braket1 = braket2 = 0
    for p in range(0,R_down_n_1*R_bose_n,R_bose_n):
        coef, function = up(i,basis_n_1[p])
        try:
            index = dict_n[''.join(map(str,function))]
            if coef != 0:
                for T in range(0,R_bose_n*R_up_n*R_down_n_1,R_bose_n*R_down_n_1):
                    braket1 += sum(Vn[index+T*R_down_n/R_down_n_1:index+R_bose_n+T*R_down_n/R_down_n_1,col1]*Vn1[p+T:p+R_bose_n+T,col2])
        except:
            continue
    for p in range(0,R_down_n*R_bose_n,R_bose_n):
        coef, function = down(j,basis_n[p])
        try:
            index = dict_n_1[''.join(map(str,function))]
            if coef != 0:
                for T in range(0,R_bose_n*R_up_n*R_down_n,R_bose_n*R_down_n):
                    braket2 += sum(Vn[p+T:p+R_bose_n+T,col1]*Vn1[index+T*R_down_n_1/R_down_n:index+R_bose_n+T*R_down_n_1/R_down_n,col2])
        except:
            continue
    return braket1*braket2
def curr_green(dict_n,dict_n_1,basis_n,basis_n_1,i,j,En,En1,Vn,Vn1,R_up_n,R_down_n,R_down_n_1,R_bose_n,beta,R,wn,sign):
    Z = sum(np.exp(-beta*En))
    G_ij = 0 + 0.j
    for nu in range(R):
        coef_p = 0
        for mu in range(R):
            coef_p += brakets(dict_n, dict_n_1,basis_n, basis_n_1, i,j,Vn,Vn1,R_up_n, R_down_n, R_down_n_1, R_bose_n,nu,mu)/((-1)**sign*(-En1[mu]+En[nu])+wn*1j)
        G_ij += np.exp(-beta*En[nu])*coef_p/Z
    return G_ij

In [57]:
def green_function(m_c, m_d, m_b, n_down, n_up, n_max, beta, U, mu, t, V, R):
    # Начальные параметры
    # базисы
    basis = (lambda n_down:bh.Basis().full_basis(m_c,m_d,m_b,n_down,n_up,n_max) )
    basis_0 = basis(n_down).astype(int)
    basis_1 = basis(n_down+1).astype(int)
    basis_2 = basis(n_down-1).astype(int)
    # Словари с.ф.
    dict_0 = create_dict(basis_0)
    dict_1 = create_dict(basis_1)
    dict_2 = create_dict(basis_2)
    # Числа состояний
    R_up_0, R_down_0, R_bose_0 = number_condition(m_c, m_d, m_b, n_down, n_up, n_max)
    R_up_1, R_down_1, R_bose_1 = number_condition(m_c, m_d, m_b, n_down+1, n_up, n_max)
    R_up_2, R_down_2, R_bose_2 = number_condition(m_c, m_d, m_b, n_down-1, n_up, n_max)
    # Вычисляем с.з. и коэффициенты разложения
    E0,V0 = eigen_values_vectors(m,n_down,n_up,n_max,U,mu,t,V)
    E1,V1 = eigen_values_vectors(m,n_down+1,n_up,n_max,U,mu,t,V)
    E2,V2 = eigen_values_vectors(m,n_down-1,n_up,n_max,U,mu,t,V)

    G = np.zeros((m_c+m_d,m_c+m_d,20),dtype=complex)

    
    for i in range(m_c+m_d,2*(m_c+m_d)):
        for j in range(m_c+m_d,2*(m_c+m_d)):
            Gn = np.zeros(20,dtype=np.complex128) # green n - 1
            GN = np.zeros(20,dtype=np.complex128) # green n + 1
            k = 0 
            for wn in bar(np.linspace(np.pi/beta,43*np.pi/beta,20 )):
                Gn[k] = curr_green(dict_0,dict_2,basis_0,basis_2,i,j,E0,E2,V0,V2,R_up_0,R_down_0,R_down_2,R_bose_0,beta,R,wn,0)
                GN[k] = curr_green(dict_1,dict_0,basis_1,basis_0,i,j,E1,E0,V1,V0,R_up_1,R_down_1,R_down_0,R_bose_1,beta,R,wn,1)
                k += 1
            G_full[i-(m_c+m_d),j-(m_c+m_d),:] = Gn + GN
    return G_full

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0., 